In [ ]:
!pip install pandas numpy pandera --quiet

### Reusable Toolkit

In [ ]:
import pandas as pd
import pandera.pandas as pa

In [ ]:
# Define a Pandera schema for our cleaned data
# This is our "contract" for what high-quality data looks like.
data_quality_schema = pa.DataFrameSchema({
    "room type": pa.Column(str, pa.Check.isin(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'])),
})

In [ ]:
# function to remove duplicates

def remove_duplicates(df):
    """Removes complete duplicate rows from a DataFrame."""
    shape_before = df.shape
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Dropped {shape_before[0] - df.shape[0]} duplicate rows.")
    return df

In [ ]:
# function to clean currency columns

def clean_currency(df, column_name, symbol='$'):
    """Cleans a currency column by removing symbols and converting to a numeric type."""
    df[column_name] = pd.to_numeric(
        df[column_name].astype(str).str.replace(symbol, '', regex=False).str.replace(',', '', regex=False),
        errors='coerce'
    )
    return df

In [ ]:
# function to impute numerical data with mean or median

def impute_numeric(df, column_name, strategy='median'):
    """Imputes a numerical column with the mean or median."""
    if strategy == 'median':
        fill_value = df[column_name].median()
    else:
        fill_value = df[column_name].mean()

    df[column_name].fillna(fill_value, inplace=True)
    return df

In [ ]:
# function to impute categorical data with mode

def impute_categorical(df, column_name):
    """ Imputes missing values in a categorical column with its mode. """
    # Calculate the mode (most frequent value)
    mode_val = df[column_name].mode()[0]
    df[column_name].fillna(mode_val, inplace=True)
    return df

In [ ]:
# function to drop columns that exceed a certain threshold of missing values

def drop_highly_missing_columns(df, threshold=0.9):
    """
    Drops columns from a DataFrame that exceed a certain threshold of missing values.
    """
    # Calculate the missing ratio for each column
    missing_ratio = df.isnull().sum() / len(df)
    # Find columns to drop
    cols_to_drop = missing_ratio[missing_ratio > threshold].index

    if len(cols_to_drop) > 0:
        print(f"Dropping columns with >{threshold*100:.0f}% missing values: {list(cols_to_drop)}")
        df = df.drop(columns=cols_to_drop)
    else:
        print(f"No columns found with more than {threshold*100:.0f}% missing values.")

    return df

In [ ]:
# A more powerful and flexible version of our text cleaning function

def standardize_text(df, columns):
    """
    Applies a standard set of text cleaning operations to a single column or a list of columns.
    """
    # First, we ensure 'columns' is a list so our loop can work correctly.
    if isinstance(columns, str):
        columns = [columns] # If it's a single string, we put it into a list.

    print(f"Standardizing text for columns: {columns}")

    # Now we can loop through the list of column names
    for col in columns:
        # Check if the column actually exists in the DataFrame to avoid errors
        if col in df.columns:
            # The cleaning logic is the same, but it's now applied to each column in the list
            df[col] = df[col].fillna('').astype(str)
            df[col] = (
                df[col].str.lower()
                       .str.strip()
                       .str.replace(r'[^\w\s]', '', regex=True) # Remove punctuation
                       .str.replace(r'[^\x00-\x7F]+', '', regex=True) # Remove emojis/non-ASCII
            )
        else:
            print(f"- Warning: Column '{col}' not found. Skipping.")

    return df

In [ ]:
# In a real project, our main script would start like this:
# from data_cleaning_utils import remove_duplicates, impute_numeric, standardize_text, ...

In [ ]:
# --- Project-Specific Handler Functions ---

def handle_uniqueness_airbnb(df):
    """Specific handler for Uniqueness in the Airbnb dataset."""
    print("Step U: Handling Uniqueness...")
    # This task is general enough that we just call our utility function directly.
    return remove_duplicates(df)

def handle_completeness_airbnb(df):
    """Specific handler for Completeness in the Airbnb dataset."""
    print("Step C: Handling Completeness...")

    # Use our reusable tool to drop columns with more than 90% missing values
    df = drop_highly_missing_columns(df, threshold=0.9)

    # This logic is specific: we only want to drop rows if 'price' or 'service fee' is missing
    df.dropna(subset=['price', 'service fee'], inplace=True)
    print(f"Dropped rows with missing critical data (price/fee).")

    # filling missing "reviews_per_month with 0"
    df['reviews per month'].fillna(0, inplace=True)

    # Use our reusable tool to impute various numeric columns
    numeric_cols = ['Construction year', 'minimum nights', 'number of reviews', 'review rate number', 'calculated host listings count',
                'availability 365']
    for col in numeric_cols:
        df = impute_numeric(df, col, strategy='median')
    print(f"Imputed numeric columns: {numeric_cols}")

    # Use our reusable tool to impute various categorical columns
    categorical_cols = ['NAME', 'host_identity_verified', 'host name',
                        'neighbourhood group', 'neighbourhood', 'country',
                        'country code', 'instant_bookable', 'cancellation_policy']

    for col in categorical_cols:
        df = impute_categorical(df, col)
    print(f"Imputed categorical columns: {categorical_cols}")

    return df

def handle_consistency_airbnb(df):
    """Specific handler for Consistency in the Airbnb dataset."""
    print("Step C: Handling Consistency...")

    # Use our reusable tool to clean the currency columns
    df = clean_currency(df, 'price', '$')
    df = clean_currency(df, 'service fee', '$')
    print("Cleaned currency columns.")

    # --- Robust Two-Pass Date Parsing for 'last review' ---
    print("Applying two-pass parsing for 'last review' dates...")

    # 1. First pass: let pandas infer most standard formats
    df['last_review_cleaned'] = pd.to_datetime(
        df['last review'],
        errors='coerce',
        infer_datetime_format=True,
        dayfirst=False
    )

    # 2. Second pass: explicitly parse the two-digit-year dates that failed the first pass
    mask = (
        df['last review'].notna() &
        df['last_review_cleaned'].isna() &
        df['last review'].str.match(r'\d{1,2}/\d{1,2}/\d{2}$')
    )
    df.loc[mask, 'last_review_cleaned'] = pd.to_datetime(
        df.loc[mask, 'last review'],
        format='%m/%d/%y',
        errors='coerce'
    )

    # 3. Clean up by dropping the original column and renaming the new one
    df.drop(columns=['last review'], inplace=True)
    df.rename(columns={'last_review_cleaned': 'last review'}, inplace=True)
    print("'last review' column parsed successfully.")

    # Standardize other text columns and fix year format
    df = standardize_text(df, ['NAME', 'host name', 'neighbourhood'])
    df['Construction year'] = df['Construction year'].astype('Int64')

    return df

def validate_data(df, schema):
    """Validates the DataFrame against a given Pandera schema."""
    print("Validating Data...")
    try:
      schema.validate(df, lazy=True) # lazy=True shows all errors at once
      print("Data validation successful. Data meets quality standards.")
    except pa.errors.SchemaErrors as err:
      print("DATA VALIDATION FAILED!")
      print(err.failure_cases)
    return df

In [ ]:
def run_airbnb_cleaning_pipeline(df):
    """
    Main pipeline function to orchestrate all Airbnb cleaning steps.
    """
    print("--- Starting Data Cleaning Pipeline ---")

    # Call each handler function in the UCCAT order
    df = handle_uniqueness_airbnb(df)
    df = handle_completeness_airbnb(df)
    df = handle_consistency_airbnb(df)
    df = validate_data(df, data_quality_schema) # Our new quality gate!

    print("--- Pipeline Finished Successfully ---")
    return df

In [ ]:
# --- Main Execution ---
print("Running the entire pipeline from raw data...\n")
raw_df = pd.read_csv('Airbnb_Data_10k.csv', low_memory=False)
cleaned_df = run_airbnb_cleaning_pipeline(raw_df.copy())

print("\n--- Final Cleaned DataFrame Info ---")
cleaned_df.info()

Running the entire pipeline from raw data...

--- Starting Data Cleaning Pipeline ---
Step U: Handling Uniqueness...
Dropped 3 duplicate rows.
Step C: Handling Completeness...
Dropping columns with >90% missing values: ['license']
Dropped rows with missing critical data (price/fee).
- Imputed numeric columns: ['Construction year', 'minimum nights', 'number of reviews', 'review rate number', 'calculated host listings count', 'availability 365']
- Imputed categorical columns: ['NAME', 'host_identity_verified', 'host name', 'neighbourhood group', 'neighbourhood', 'country', 'country code', 'instant_bookable', 'cancellation_policy']
Step C: Handling Consistency...
- Cleaned currency columns.
- Applying two-pass parsing for 'last review' dates...


/tmp/ipython-input-11-3591929316.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews per month'].fillna(0, inplace=True)
/tmp/ipython-input-6-2797534038.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

   - 'last review' column parsed successfully.
Standardizing text for columns: ['NAME', 'host name', 'neighbourhood']
Validating Data...
Data validation successful. Data meets quality standards.
--- Pipeline Finished Successfully ---

--- Final Cleaned DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 9948 entries, 0 to 9996
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              9948 non-null   int64         
 1   NAME                            9948 non-null   object        
 2   host id                         9948 non-null   int64         
 3   host_identity_verified          9948 non-null   object        
 4   host name                       9948 non-null   object        
 5   neighbourhood group             9948 non-null   object        
 6   neighbourhood                   9948 non-null   object        
 7   lat      

In [ ]:
cleaned_df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,last review
0,27623301,nyc oasis in gramercy,31250838867,unconfirmed,michal,Manhattan,stuyvesant town,40.73549,-73.97707,United States,...,2021,951,190,20.0,0.0,0.00,4.0,1.0,30.0,NaT
1,53024130,gorgeous times square flat,46261863080,verified,jowelle,Manhattan,hells kitchen,40.76008,-73.98846,United States,...,2021,543,109,2.0,41.0,2.79,5.0,5.0,169.0,2019-05-29
2,50922076,great room host steps from 1 train,17623893479,unconfirmed,dee,Manhattan,harlem,40.82378,-73.95354,United States,...,2010,812,162,60.0,13.0,0.67,3.0,4.0,0.0,2018-08-15
3,50265391,spacious private bedroom in the north bronx,48334233422,verified,liliana,Bronx,kingsbridge,40.88283,-73.89326,United States,...,2006,328,66,3.0,2.0,0.07,5.0,1.0,363.0,2017-05-25
4,5566648,colorful spacious near a express,45624334759,verified,andrew,Manhattan,washington heights,40.85111,-73.93325,United States,...,2015,132,26,2.0,48.0,0.97,4.0,1.0,223.0,2019-06-30
